# Prompt engineering

Context engineering is the practice of crafting the information that flows into and out of AI agents. Among the various "write strategies" for producing high-quality context, prompt engineering stands as one of the most fundamental. Well-designed prompts serve as the blueprint for agent behavior, guiding not just what the agent says, but how it reasons, what information it prioritizes, and how it structures its responses.

The challenge is finding the right level of detail and organization. Prompts that are too vague leave agents directionless, producing inconsistent or off-target outputs. Prompts that are too prescriptive become brittle and token-inefficient, micromanaging every detail at the cost of flexibility. The solution lies in treating prompts as composable, reusable modules that can be mixed and matched based on task requirements, enhanced with strategic examples, and structured with clear delimiters to maximize both clarity and efficiency.

In this notebook, we explore systematic techniques for engineering prompts that produce consistent, high-quality outputs while minimizing token waste. We will examine how to structure system prompts as composable modules, use few-shot examples strategically to guide behavior, find the appropriate "altitude" for instructions, and leverage structured delimiters to organize information clearly.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

We begin by initializing the language model that will process our prompts. Using a consistent model with temperature 0 ensures deterministic, reproducible outputs across our examples.

In [2]:
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    api_key=os.getenv("OPENAI_API_KEY", "").strip(),
    temperature=0  # Set to 0 for more deterministic outputs
)

## Part 1: Composable prompt modules

The traditional approach to prompt engineering creates monolithic prompts—single, large blocks of text that attempt to handle every possible scenario. While this might seem comprehensive, it quickly becomes problematic as systems grow. These monolithic prompts load all information into every request regardless of relevance, wasting tokens and diluting focus. They become maintenance nightmares as different sections need updates at different times and they cannot be reused across different agents or tasks.

The solution is composability: breaking prompts into focused, reusable modules that can be mixed and matched based on task requirements. Each module handles a specific concern - identity, communication style, product information, policies, security rules - and can be independently maintained and tested. When a request comes in, we select only the relevant modules, dramatically reducing token usage while improving clarity and maintainability.

In [3]:
# Example of a monolithic prompt that tries to handle everything at once
monolithic_prompt = """You are a customer service agent for TechStore.

RESPONSIBILITIES:
- Answer product questions
- Process orders and returns
- Handle technical support
- Manage account issues
- Provide shipping information

GUIDELINES:
- Be professional and courteous
- Keep responses under 3 sentences
- Always verify customer identity for account changes
- Escalate billing issues to finance team
- Never share customer data with third parties

PRODUCTS:
- Laptop Pro X1: $1299, 16GB RAM, 512GB SSD
- Tablet Mini: $499, 10-inch screen, 128GB
- Wireless Earbuds: $199, noise cancelling

POLICIES:
- 30-day return policy
- Free shipping over $50
- 2-year warranty on electronics
- Price matching within 14 days

Customer question: {question}"""

# Test with a simple question
question = "How much is the Laptop Pro X1?"
response = llm.invoke(monolithic_prompt.format(question=question))

print("Question:", question)
print("\nResponse with monolithic prompt:")
print(response.content)
print(f"\nPrompt length: {len(monolithic_prompt.format(question=question))} characters")

Question: How much is the Laptop Pro X1?

Response with monolithic prompt:
The Laptop Pro X1 is priced at $1299. If you have any other questions or need assistance, feel free to ask!

Prompt length: 743 characters


This example demonstrates the monolithic approach in action:
1. Defines a single large prompt containing all possible information including responsibilities, guidelines, product catalog, and policies.
2. Formats the prompt with a simple product pricing question.
3. Sends the entire prompt to the model even though most sections are irrelevant to this specific query.
4. Displays both the response and the total character count, highlighting the inefficiency.

The prompt contains 500+ characters of information when only about 100 characters (product catalog section) are actually needed to answer the question. This wastes tokens and makes the model process irrelevant context.

In [4]:
# Module 1: Core identity - defines the agent's basic role
IDENTITY_MODULE = """You are a customer service agent for TechStore."""

# Module 2: Communication guidelines - defines interaction style
COMMUNICATION_MODULE = """Communication style:
- Be professional and courteous
- Keep responses concise and focused
- Show empathy for customer concerns"""

# Module 3: Product catalog - for product-related questions
PRODUCT_CATALOG_MODULE = """Product information:
- Laptop Pro X1: $1299, 16GB RAM, 512GB SSD
- Tablet Mini: $499, 10-inch screen, 128GB
- Wireless Earbuds: $199, noise cancelling"""

# Module 4: Store policies - for returns, shipping, warranty questions
POLICY_MODULE = """Store policies:
- 30-day return policy
- Free shipping over $50
- 2-year warranty on electronics
- Price matching within 14 days"""

# Module 5: Security requirements - for account operations
SECURITY_MODULE = """Security requirements:
- Always verify customer identity before account changes
- Never share customer data with third parties
- Escalate billing issues to finance team"""

print("Modular components created successfully!")
print(f"\nTotal modules: 5")
print(f"Identity module: {len(IDENTITY_MODULE)} chars")
print(f"Communication module: {len(COMMUNICATION_MODULE)} chars")
print(f"Product module: {len(PRODUCT_CATALOG_MODULE)} chars")
print(f"Policy module: {len(POLICY_MODULE)} chars")
print(f"Security module: {len(SECURITY_MODULE)} chars")

Modular components created successfully!

Total modules: 5
Identity module: 47 chars
Communication module: 126 chars
Product module: 150 chars
Policy module: 128 chars
Security module: 168 chars


Having broken down the monolithic prompt into modules, we can now compose task-specific prompts by selecting only the relevant pieces. This function demonstrates dynamic prompt composition based on task type.

In [5]:
def compose_prompt(task_type: str, question: str) -> str:
    """Compose a prompt by selecting relevant modules based on task type.
    
    Args:
        task_type: Type of task (product_question, policy_question, etc.)
        question: Customer's question
        
    Returns:
        Composed prompt string with only relevant modules
    """
    
    # All tasks get identity and communication modules (always needed)
    modules = [IDENTITY_MODULE, COMMUNICATION_MODULE]
    
    # Add task-specific modules based on what the customer needs
    if task_type == "product_question":
        modules.append(PRODUCT_CATALOG_MODULE)
    elif task_type == "policy_question":
        modules.append(POLICY_MODULE)
    elif task_type == "account_change":
        modules.extend([SECURITY_MODULE, POLICY_MODULE])
    elif task_type == "purchase":
        modules.extend([PRODUCT_CATALOG_MODULE, POLICY_MODULE])
    
    # Compose final prompt by joining selected modules
    prompt = "\n\n".join(modules)
    prompt += f"\n\nCustomer question: {question}"
    
    return prompt

# Test with different task types to demonstrate selective module loading
test_cases = [
    ("product_question", "How much is the Laptop Pro X1?"),
    ("policy_question", "What's your return policy?"),
]

for task_type, question in test_cases:
    print(f"\n{'='*60}")
    print(f"Task: {task_type}")
    print(f"Question: {question}")
    print(f"{'='*60}")
    
    # Compose prompt with only necessary modules
    composed_prompt = compose_prompt(task_type, question)
    print(f"Composed prompt ({len(composed_prompt)} chars):")
    print(composed_prompt)
    
    # Get response from model
    response = llm.invoke(composed_prompt)
    print(f"\nResponse:\n{response.content}")


Task: product_question
Question: How much is the Laptop Pro X1?
Composed prompt (378 chars):
You are a customer service agent for TechStore.

Communication style:
- Be professional and courteous
- Keep responses concise and focused
- Show empathy for customer concerns

Product information:
- Laptop Pro X1: $1299, 16GB RAM, 512GB SSD
- Tablet Mini: $499, 10-inch screen, 128GB
- Wireless Earbuds: $199, noise cancelling

Customer question: How much is the Laptop Pro X1?

Response:
The Laptop Pro X1 is priced at $1,299. If you have any other questions or need further assistance, feel free to ask!

Task: policy_question
Question: What's your return policy?
Composed prompt (352 chars):
You are a customer service agent for TechStore.

Communication style:
- Be professional and courteous
- Keep responses concise and focused
- Show empathy for customer concerns

Store policies:
- 30-day return policy
- Free shipping over $50
- 2-year warranty on electronics
- Price matching within 14 days

Cus

## Part 2: Strategic few-shot examples

Few-shot examples are one of the most powerful tools in prompt engineering. By showing the model concrete examples of desired behavior, we can guide it toward specific output formats, reasoning patterns, and quality standards without lengthy explanations. The key is using examples strategically - providing just enough to establish the pattern without cluttering the context window. Well-chosen examples teach the model through demonstration rather than prescription, making prompts both clearer and more token-efficient.

The effectiveness of few-shot learning lies in its ability to show rather than tell. Instead of describing in detail what we want, we provide 2-3 representative examples that demonstrate the desired behavior. This is particularly valuable for complex formatting requirements, domain-specific reasoning patterns and edge case handling where natural language descriptions would be verbose or ambiguous.

In [6]:
# Zero-shot: No examples provided, relying on model's general capabilities
zero_shot_prompt = """Extract the product name, price, and category from this text.

Text: The Laptop Pro X1 costs $1299 and is in the computers category.

Output:"""

# Invoke the model without any examples to guide it
response = llm.invoke(zero_shot_prompt)
print("Zero-shot response:")
print(response.content)
print()

Zero-shot response:
Product Name: Laptop Pro X1  
Price: $1299  
Category: Computers



The zero-shot approach often produces reasonable results, but we have limited control over the output format. The model might choose any structure - natural language sentences, bullet points or some other format. For production systems requiring consistent, parseable outputs, this variability is problematic. Few-shot examples solve this by establishing a clear format through demonstration.

In [7]:
# Few-shot: Include examples of desired behavior to establish format
few_shot_prompt = """Extract the product name, price, and category from text. Format as JSON.

Example 1:
Text: The Wireless Mouse is priced at $29.99 and belongs to accessories.
Output: {"product": "Wireless Mouse", "price": 29.99, "category": "accessories"}

Example 2:
Text: Get the 4K Monitor for only $399 in the displays section.
Output: {"product": "4K Monitor", "price": 399.00, "category": "displays"}

Now extract from this:
Text: The Laptop Pro X1 costs $1299 and is in the computers category.

Output:"""

# Invoke with examples guiding the output format
response = llm.invoke(few_shot_prompt)
print("Few-shot response:")
print(response.content)

Few-shot response:
{"product": "Laptop Pro X1", "price": 1299.00, "category": "computers"}


Now the model consistently returns well-formatted JSON that can be parsed programmatically. This demonstrates how few-shot examples guide not just the content but also the structure of outputs. Two examples were sufficient to establish the pattern - showing the value of quality over quantity in few-shot learning.

In [8]:
# Few-shot for reasoning patterns - show how to approach the problem
reasoning_prompt = """Determine if a customer is eligible for a refund based on our 30-day policy.

Example 1:
Purchase date: 2024-01-15
Refund request date: 2024-02-01
Days elapsed: 17 days
Decision: APPROVED - Within 30-day window

Example 2:
Purchase date: 2024-01-01  
Refund request date: 2024-02-15
Days elapsed: 45 days
Decision: DENIED - Exceeds 30-day policy

Example 3:
Purchase date: 2024-02-05
Refund request date: 2024-02-29
Days elapsed: 24 days
Decision: APPROVED - Within 30-day window

Now evaluate this case:
Purchase date: 2024-01-20
Refund request date: 2024-02-25

Analysis:"""

# Invoke with examples demonstrating the reasoning process
response = llm.invoke(reasoning_prompt)
print("Reasoning with few-shot examples:")
print(response.content)

Reasoning with few-shot examples:
To determine if the customer is eligible for a refund based on the 30-day policy, we need to calculate the number of days elapsed between the purchase date and the refund request date.

**Purchase date:** 2024-01-20  
**Refund request date:** 2024-02-25  

Now, let's calculate the days elapsed:

1. From January 20 to January 31: 11 days (31 - 20 = 11)
2. From February 1 to February 25: 25 days

Now, we add these two periods together:

11 days (January) + 25 days (February) = 36 days elapsed.

Since 36 days exceeds the 30-day refund policy, the decision is:

**Decision: DENIED - Exceeds 30-day policy**


### Best practices for few-shot examples

The quality and selection of few-shot examples significantly impact their effectiveness. Rather than providing many examples, focus on 2-3 high-quality ones that cover the most important patterns. Include edge cases that might cause confusion, maintain consistent formatting across all examples, and ensure examples represent the diversity of scenarios the agent will encounter. This strategic approach maximizes learning while minimizing token usage.

In [9]:
# Creating a few-shot module for sentiment analysis with action recommendations
FEW_SHOT_SENTIMENT = """Analyze customer feedback sentiment and categorize the issue.

Example 1:
Feedback: "The laptop arrived quickly and works great! Very happy with my purchase."
Sentiment: Positive
Category: Product Quality
Action: None needed

Example 2:
Feedback: "Shipping took forever and the box was damaged. The product is fine though."
Sentiment: Mixed
Category: Shipping
Action: Review shipping partner

Example 3:
Feedback: "This is the worst purchase I've ever made. It broke after 2 days!"
Sentiment: Negative
Category: Product Defect
Action: Immediate replacement + apology
"""

# Test the few-shot pattern with new feedback
test_feedback = "The customer service was amazing but the product quality is disappointing."
prompt = FEW_SHOT_SENTIMENT + f"\nNew feedback: {test_feedback}\n\nAnalysis:"

response = llm.invoke(prompt)
print("Sentiment analysis with few-shot examples:")
print(response.content)

Sentiment analysis with few-shot examples:
Feedback: "The customer service was amazing but the product quality is disappointing."
Sentiment: Mixed
Category: Customer Service and Product Quality
Action: Address product quality issues and acknowledge positive customer service feedback


## Part 3: Altitude-appropriate prompts

Prompt "altitude" refers to the level of detail and specificity in our instructions. Finding the right altitude is crucial: too high (vague) and the agent lacks direction, producing inconsistent outputs; too low (prescriptive) and the agent becomes rigid, unable to adapt to varying contexts. The optimal altitude depends on task complexity, the model's capabilities, the acceptable level of output variance, and the consequences of errors.

The goal is to provide enough structure to ensure reliable behavior without over-constraining the agent. High-stakes operations like security verification require lower altitude with detailed step-by-step instructions. Creative tasks or domains where the model has strong knowledge benefit from higher altitude, allowing natural adaptation to context. Most production systems need balanced altitude—clear guidance on what matters while preserving flexibility in execution.

In [10]:
# Too vague - gives model excessive freedom without clear direction
vague_prompt = "Help the customer."

question = "I want to return my laptop."
response = llm.invoke(f"{vague_prompt}\n\nCustomer: {question}\n\nAgent:")
print("Too vague prompt response:")
print(response.content)
print()

Too vague prompt response:
Of course! I can help you with that. Could you please provide me with your order number and the reason for the return? Additionally, let me know if the laptop is still in its original packaging and if you have all the accessories that came with it.



While the vague prompt might produce helpful responses, we sacrifice control over critical elements like tone consistency, which information to request, and which policies to mention. Different runs might produce widely varying responses, making the system unpredictable and difficult to test or maintain.

In [11]:
# Too prescriptive - micromanages every detail and interaction step
prescriptive_prompt = """You are a returns specialist. When a customer wants to return something:
1. First, greet them warmly using exactly these words: "I'd be happy to help with your return."
2. Then ask for their order number using this exact phrasing: "Could you please provide your order number?"
3. After they provide it, say: "Thank you for providing that information."
4. Then ask: "May I know the reason for the return?"
5. After they answer, explain the 30-day policy using these words: "Our return policy allows returns within 30 days of purchase."
6. Then ask: "When did you receive the item?"
7. Calculate the days and inform them if eligible.
8. End with: "Is there anything else I can help you with today?"

Follow these steps exactly in this order."""

question = "I want to return my laptop."
response = llm.invoke(f"{prescriptive_prompt}\n\nCustomer: {question}\n\nAgent:")
print("Too prescriptive prompt response:")
print(response.content)
print()

Too prescriptive prompt response:
I'd be happy to help with your return. Could you please provide your order number?



This level of prescription creates robotic, inflexible interactions. The agent cannot adapt to context, handle interruptions gracefully, or provide a natural conversational experience. It's also token-inefficient, using many words to specify details that could be left to the model's judgment.

In [12]:
# Appropriate altitude - clear guidance with flexibility for natural adaptation
balanced_prompt = """You are a returns specialist for TechStore.

When handling return requests:
- Acknowledge the request warmly
- Ask for order number and purchase date
- Explain our 30-day return policy
- Verify eligibility based on purchase date
- Provide next steps if approved, or alternatives if denied

Maintain a helpful, professional tone throughout."""

question = "I want to return my laptop."
response = llm.invoke(f"{balanced_prompt}\n\nCustomer: {question}\n\nAgent:")
print("Appropriate altitude prompt response:")
print(response.content)

Appropriate altitude prompt response:
Hello! Thank you for reaching out to us about your return request. I’m here to help you with that. Could you please provide me with your order number and the purchase date of your laptop? 

Just to let you know, we have a 30-day return policy, so I’ll verify your eligibility based on the purchase date once I have that information. Looking forward to assisting you further!


### Guidelines for choosing the right altitude

The decision between high and low altitude prompts depends on specific requirements and constraints. Use lower altitude with more prescriptive detail when output format is critical for API integration or structured data processing, when compliance or legal requirements must be met exactly, when errors carry high consequences, or when the agent is operating in an unfamiliar domain. Conversely, use higher altitude with less prescriptive detail when creative or varied responses add value, when the agent needs to adapt dynamically to diverse contexts, when the domain is well within the model's training, or when variety in tone and approach is desirable.

In [13]:
# Example: Choosing altitude based on task requirements and risk profile

# High-stakes task: Lower altitude (more prescriptive) for security operations
security_prompt = """You are a security verification agent.

Before any account changes, you MUST:
1. Verify identity using exactly two factors:
   - Last 4 digits of registered phone number
   - Account holder's date of birth
2. Ask each question separately, wait for response
3. Do NOT proceed unless both match our records
4. If verification fails, immediately escalate to security team
5. Never share what information we have on file

Do not deviate from this protocol."""

print("Low altitude (prescriptive) for security:")
print(security_prompt)
print("\n" + "="*60 + "\n")

# Low-stakes task: Higher altitude (more flexible) for general assistance
chat_prompt = """You are a friendly shopping assistant.

Help customers discover products they'll love by:
- Asking about their needs and preferences
- Suggesting relevant products
- Answering questions enthusiastically

Be conversational and adapt to their shopping style."""

print("High altitude (flexible) for general chat:")
print(chat_prompt)

Low altitude (prescriptive) for security:
You are a security verification agent.

Before any account changes, you MUST:
1. Verify identity using exactly two factors:
   - Last 4 digits of registered phone number
   - Account holder's date of birth
2. Ask each question separately, wait for response
3. Do NOT proceed unless both match our records
4. If verification fails, immediately escalate to security team
5. Never share what information we have on file

Do not deviate from this protocol.


High altitude (flexible) for general chat:
You are a friendly shopping assistant.

Help customers discover products they'll love by:
- Asking about their needs and preferences
- Suggesting relevant products
- Answering questions enthusiastically

Be conversational and adapt to their shopping style.


## Part 4: Structured delimiters

Structured delimiters organize information within prompts, making complex content easier for both models and humans to parse. Without clear delimiters, prompts become walls of text where critical information blends into surrounding context. Delimiters create visual and logical boundaries that help models identify sections, prioritize information, and understand relationships between different parts of the prompt.

Common delimiter approaches include section headers using markdown or symbols, XML-style tags that create clear hierarchical structure, and simple labels that mark distinct information types. The choice of delimiter style should match the complexity of your content - simple labels work well for flat structures, markdown excels at human readability, and XML-style tags shine for complex nested information.

In [14]:
# Unstructured prompt - difficult to parse and locate specific information
unstructured = """You are a tech support agent. The customer has a laptop that won't turn on. 
First check if it's plugged in. Then check the battery. Then check for indicator lights. 
Our laptops have LED indicators: green means charging, amber means battery low, red means 
hardware issue, no light means no power. Always ask one question at a time. Be patient and 
clear. Never assume the customer has technical knowledge.

Customer says: My laptop won't start"""

response = llm.invoke(unstructured)
print("Response without delimiters:")
print(response.content)
print()

Response without delimiters:
I'm sorry to hear that your laptop isn't starting. Let's go through some steps to troubleshoot the issue. 

First, can you please check if your laptop is plugged in? Make sure the power adapter is connected to both the laptop and a working electrical outlet. What do you see?



The unstructured prompt contains all necessary information but presents it as a continuous block of text. This makes it difficult to scan quickly, hard to identify where one concept ends and another begins, and challenging to update specific sections without accidentally affecting others. Structured delimiters solve these problems.

In [15]:
# Well-structured prompt with clear section delimiters
structured = """### ROLE
You are a tech support agent helping customers with laptop issues.

### TROUBLESHOOTING PROCEDURE
1. Check if laptop is plugged into power
2. Check battery status
3. Check LED indicator lights

### LED INDICATORS
- Green: Charging normally
- Amber: Battery low
- Red: Hardware issue detected
- No light: No power received

### INTERACTION GUIDELINES
- Ask one question at a time
- Be patient and clear
- Don't assume technical knowledge

### CUSTOMER ISSUE
"My laptop won't start"

### YOUR RESPONSE
"""

response = llm.invoke(structured)
print("Response with clear delimiters:")
print(response.content)

Response with clear delimiters:
I'm sorry to hear that your laptop won't start. Let's try to troubleshoot the issue together. 

First, can you please check if your laptop is plugged into a power source?


The structured version organizes identical information into clearly labeled sections. Models can quickly locate relevant information, humans can update specific sections without errors, and the logical flow from role to procedure to guidelines becomes immediately apparent.

In [16]:
# Style 1: XML-style tags provide explicit structure
xml_style = """<role>
Customer service agent for TechStore
</role>

<context>
Customer purchased Laptop Pro X1 30 days ago for $1299
</context>

<instructions>
- Determine refund eligibility
- Explain decision clearly
- Offer alternatives if denied
</instructions>

<customer_message>
I'd like to return this laptop.
</customer_message>

<response>
"""

print("XML-style delimiters:")
print(xml_style)
print("\n" + "="*60 + "\n")

# Style 2: Markdown headers for readability
markdown_style = """# Role
Customer service agent for TechStore

## Context
Customer purchased Laptop Pro X1 30 days ago for $1299

## Instructions
- Determine refund eligibility
- Explain decision clearly  
- Offer alternatives if denied

## Customer Message
"I'd like to return this laptop."

## Your Response
"""

print("Markdown-style delimiters:")
print(markdown_style)
print("\n" + "="*60 + "\n")

# Style 3: Simple labels for straightforward content
label_style = """ROLE: Customer service agent for TechStore

CONTEXT: Customer purchased Laptop Pro X1 30 days ago for $1299

INSTRUCTIONS:
- Determine refund eligibility
- Explain decision clearly
- Offer alternatives if denied

CUSTOMER: "I'd like to return this laptop."

AGENT:"""

print("Label-style delimiters:")
print(label_style)

XML-style delimiters:
<role>
Customer service agent for TechStore
</role>

<context>
Customer purchased Laptop Pro X1 30 days ago for $1299
</context>

<instructions>
- Determine refund eligibility
- Explain decision clearly
- Offer alternatives if denied
</instructions>

<customer_message>
I'd like to return this laptop.
</customer_message>

<response>



Markdown-style delimiters:
# Role
Customer service agent for TechStore

## Context
Customer purchased Laptop Pro X1 30 days ago for $1299

## Instructions
- Determine refund eligibility
- Explain decision clearly  
- Offer alternatives if denied

## Customer Message
"I'd like to return this laptop."

## Your Response



Label-style delimiters:
ROLE: Customer service agent for TechStore

CONTEXT: Customer purchased Laptop Pro X1 30 days ago for $1299

INSTRUCTIONS:
- Determine refund eligibility
- Explain decision clearly
- Offer alternatives if denied

CUSTOMER: "I'd like to return this laptop."

AGENT:


All three styles effectively organize the same information:
1. XML tags work best for complex nested structures where hierarchy matters.
2. Markdown headers excel at human readability and are familiar to most developers.
3. Simple labels provide the most concise approach for flat, straightforward structures.

Choose based on your content complexity and team preferences.

In [17]:
# Well-structured prompt combining delimiters with few-shot examples
comprehensive_prompt = """### ROLE
Order status assistant for TechStore

### TASK
Parse order status updates and provide customer-friendly summaries.

### EXAMPLES

Example 1:
---
Input: {"order_id": "12345", "status": "shipped", "tracking": "ABC123", "eta": "2024-02-20"}
Output: "Great news! Your order #12345 has shipped and is on its way. Track it with ABC123. Expected delivery: February 20, 2024."
---

Example 2:
---
Input: {"order_id": "67890", "status": "delayed", "reason": "weather", "new_eta": "2024-02-25"}
Output: "We apologize for the delay on order #67890. Due to weather conditions, your new expected delivery is February 25, 2024."
---

### NEW INPUT
{"order_id": "11111", "status": "delivered", "signed_by": "John Doe", "date": "2024-02-18"}

### YOUR OUTPUT
"""

response = llm.invoke(comprehensive_prompt)
print("Comprehensive structured prompt response:")
print(response.content)

Comprehensive structured prompt response:
"Your order #11111 has been successfully delivered! It was signed for by John Doe on February 18, 2024. Thank you for shopping with us!"


## Putting it all together

Production prompts require combining multiple best practices into a cohesive system. Rather than choosing a single technique, effective prompt engineering layers composable modules, strategic examples, appropriate altitude and structured delimiters to create prompts that are both powerful and maintainable. This integrated approach ensures prompts remain clear as complexity grows.

In [18]:
class PromptBuilder:
    """Build production-ready prompts using best practices.
    
    This class demonstrates how to combine multiple prompt engineering techniques:
    - Composable modules for token efficiency
    - Few-shot examples for format guidance
    - Altitude-appropriate instructions
    - Structured delimiters for clarity
    """
    
    def __init__(self):
        # Core reusable modules
        self.modules = {
            "identity": "You are a customer service agent for TechStore.",
            "communication": """### COMMUNICATION STYLE
- Professional and courteous
- Clear and concise
- Empathetic to customer needs""",
            "products": """### PRODUCTS
- Laptop Pro X1: $1299, 16GB RAM, 512GB SSD
- Tablet Mini: $499, 10-inch screen, 128GB
- Wireless Earbuds: $199, noise cancelling""",
            "policies": """### POLICIES
- 30-day return policy
- Free shipping over $50
- 2-year warranty on electronics"""
        }
        
        # Few-shot examples organized by task type
        self.examples = {
            "refund": """### EXAMPLES

Example 1:
Customer: "I bought this 20 days ago and want a refund."
Agent: "I'd be happy to help with your refund. Since you're within our 30-day window, you're eligible. I'll need your order number to process this."

Example 2:
Customer: "I need to return something I bought 40 days ago."
Agent: "I understand you'd like a return. Unfortunately, this falls outside our 30-day policy. However, I can check if the warranty covers your issue or offer store credit."
""",
            "product_info": """### EXAMPLES

Example:
Customer: "Tell me about the Laptop Pro X1."
Agent: "The Laptop Pro X1 is $1299 and features 16GB RAM with 512GB SSD storage. It includes our 2-year warranty and qualifies for free shipping."
"""
        }
    
    def build_prompt(self, task_type: str, customer_message: str, 
                    include_examples: bool = True, altitude: str = "balanced") -> str:
        """Build a prompt using composable modules and best practices.
        
        Args:
            task_type: Type of task (refund, product_info, etc.)
            customer_message: The customer's message to respond to
            include_examples: Whether to include few-shot examples
            altitude: Instruction detail level (prescriptive, balanced, flexible)
            
        Returns:
            Complete prompt string ready for the model
        """
        
        sections = []
        
        # 1. Identity (always included)
        sections.append(self.modules["identity"])
        
        # 2. Communication style (always included)
        sections.append(self.modules["communication"])
        
        # 3. Task-specific modules
        if task_type in ["product_info", "purchase"]:
            sections.append(self.modules["products"])
        
        if task_type in ["refund", "policy", "purchase"]:
            sections.append(self.modules["policies"])
        
        # 4. Few-shot examples (if requested and available)
        if include_examples and task_type in self.examples:
            sections.append(self.examples[task_type])
        
        # 5. Instructions (altitude-appropriate)
        if altitude == "prescriptive":
            instructions = """### INSTRUCTIONS
1. Read the customer message carefully
2. Identify the specific need
3. Check relevant policies or product info
4. Formulate a response that addresses the need
5. Keep response under 3 sentences"""
            sections.append(instructions)
        elif altitude == "flexible":
            instructions = """### INSTRUCTIONS
Address the customer's needs appropriately."""
            sections.append(instructions)
        # balanced altitude adds no extra instructions
        
        # 6. Customer message with clear delimiter
        sections.append(f"""### CUSTOMER MESSAGE
{customer_message}

### YOUR RESPONSE""")
        
        return "\n\n".join(sections)

# Test the prompt builder with a real scenario
builder = PromptBuilder()

# Test case: Refund request
prompt1 = builder.build_prompt(
    task_type="refund",
    customer_message="I bought a laptop 25 days ago and want to return it.",
    include_examples=True,
    altitude="balanced"
)

print("Generated Prompt:")
print("="*60)
print(prompt1)
print("="*60)

response = llm.invoke(prompt1)
print("\nAgent Response:")
print(response.content)

Generated Prompt:
You are a customer service agent for TechStore.

### COMMUNICATION STYLE
- Professional and courteous
- Clear and concise
- Empathetic to customer needs

### POLICIES
- 30-day return policy
- Free shipping over $50
- 2-year warranty on electronics

### EXAMPLES

Example 1:
Customer: "I bought this 20 days ago and want a refund."
Agent: "I'd be happy to help with your refund. Since you're within our 30-day window, you're eligible. I'll need your order number to process this."

Example 2:
Customer: "I need to return something I bought 40 days ago."
Agent: "I understand you'd like a return. Unfortunately, this falls outside our 30-day policy. However, I can check if the warranty covers your issue or offer store credit."


### CUSTOMER MESSAGE
I bought a laptop 25 days ago and want to return it.

### YOUR RESPONSE

Agent Response:
I'd be happy to assist you with your return. Since you're within our 30-day return policy, you're eligible for a refund. Please provide your or

By applying these techniques systematically, we create prompts that are token-efficient, guide models effectively toward desired outputs, remain maintainable as systems grow in complexity and produce consistent high-quality results. This foundation is essential for production AI systems where prompt quality directly impacts agent performance and reliability.